In [3]:
import pandas as pd
import numpy as np

In [4]:
filepath='test_data/user_following_animation.json'
data=pd.read_json(filepath,lines=True)

In [61]:
user_info = pd.read_csv("test_data/bilibili_crawler_user_info.csv",names = ['id','mid','name','sex','sign','the_rank','level','jointime','moral','silence','birthday','coins','fans_badge','role','title','desc','vip_type','vip_status'])

In [62]:
user_info.head()

,id,mid,name,sex,sign,the_rank,level,jointime,moral,silence,birthday,coins,fans_badge,role,title,desc,vip_type,vip_status
0,1,17515174,试图让群主女装,NaN,NaN,10000.0,5,0.0,0.0,0.0,02-24,0.0,0.0,0.0,NaN,NaN,2.0,1.0
1,2,7596516,酒客小丑,NaN,沉迷于吃狗粮中。商业合作请加QQ2375432747 微博：@酒客小丑,10000.0,6,0.0,0.0,0.0,06-15,0.0,1.0,1.0,bilibili 2018百大UP主、高能联盟成员,NaN,2.0,1.0
2,3,10769575,文史阁,NaN,地势坤，君子以厚德载物。,10000.0,6,0.0,0.0,0.0,01-01,0.0,0.0,1.0,bilibili 知名科普UP主,代表作：中国历代帝王合集,0.0,0.0
3,4,22989278,果冻萌萌的,NaN,林正英以及香港经典鬼片僵尸片24小时播放 房间号：193 欢迎大家观看,10000.0,6,0.0,0.0,0.0,09-17,0.0,1.0,1.0,bilibili 知名UP主,NaN,1.0,0.0
4,5,130050712,初阳手绘石头,NaN,如果搜不到我，说明你是在查找你的好友里搜，当然搜不到了。因为我还不是你的好友呢，你要在添＋好...,10000.0,5,0.0,0.0,0.0,01-01,0.0,1.0,0.0,NaN,NaN,2.0,1.0


## user_info 预处理

In [67]:
user_info[user_info.vip_type.isna() | user_info.the_rank.isna() | user_info.level.isna()] # 看一下vip_type、the_rank、level有NaN的值

,id,mid,name,sex,sign,the_rank,level,jointime,moral,silence,birthday,coins,fans_badge,role,title,desc,vip_type,vip_status
2534,2535,5398071,吾谶,NaN,CP可以逆 本命不能移,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2535,NaN,10000,5,0.0,0,0.0,10-16,0.0,0.0,0.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN
21457,21457,177589181,哔哩哔哩安全应急响应中心,NaN,"><img src=1 onerror=(alert)(2233)>,10000,3,0,0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21458,定期搬运各种科普游戏视频，欢迎关注；想来我们不正规字幕组或唠嗑的小伙伴，请加入交流QQ群：7...,10000,6,0.0,0,0.0,05-05,0.0,1.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
39013,39020,77818,雾香·楼兰,NaN,之前账号：科洛丝琳希停用,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39014,NaN,10000,5,0.0,0,1.0,NaN,0.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
44168,44174,402348056,苍蓝誓约手游,NaN,初次见面！您...就是指挥官大人吗？emmm，跟想象中不太一样呢~总之，请多多指教喽，指挥官...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44169,"最近迷上做神踩点视频""",10000,3,0.0,0,0.0,03-15,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [68]:
user_info[user_info.vip_type.isna() | user_info.the_rank.isna() | user_info.level.isna()].index.tolist()

[2534, 2535, 21457, 21458, 39013, 39014, 44168, 44169]

In [70]:
user_info.drop(user_info[user_info.vip_type.isna() | user_info.the_rank.isna() | user_info.level.isna()].index.tolist(),inplace=True) # 将vip_type、the_rank、level有NaN的行去掉

## 定义权重规则

In [156]:
# 看一下这三个可能的取值
print('the_rank:',set(user_info.the_rank))
print('level:',set(user_info.level))
print('vip_type:',set(user_info.vip_type))

the_rank: {10000.0, 30000.0, 20000.0, 25000.0}
level: {'6', '3', '4', '5'}
vip_type: {0.0, 1.0, 2.0}


In [89]:
rules={'the_rank':{10000:1,20000:2,25000:3,30000:4},'level':{'3':1,'4':2,'5':3,'6':4},'vip_type':{0:0,1:1,2:2}}


In [157]:
# 检查一下 user_info.mid有没有重的
print(len(user_info))
print(len(set(user_info.mid)))
a=set()
for index,row in user_info.iterrows():
    if row.mid in a:
        print()

        print(row)
    else:
        a.add(row.mid)
            

51424
51424


## 根据权重增加数据函数

In [159]:
def user_power(data,rules,user_info):
    """
    params:
        data:用户收藏ID数据集
        rules:权重规则
        user_info:用户信息数据集
    return:新数据集
        
    """
    new_data = pd.DataFrame(columns=data.columns)
    for index,row in data.iterrows():
        info = user_info[user_info.mid == int(row.key)]
        if len(info):
            the_power = rules['the_rank'][info.the_rank.values[0]] + rules['level'][info.level.values[0]] + rules['vip_type'][info.vip_type.values[0]]
        else:  # len(info)==0 说明在user_info中没有这个用户的相关数据
            the_power = 1
#         print(the_power)
        for i in range(the_power):
            new_data = new_data.append(row,ignore_index=True)
    
    return new_data
    

### 函数测试

In [163]:
# 测试数据集
test_data = data.head() 
test_data

,db,key,size,ttl,type,value
0,7,330817737,6,-1,set,[132112]
1,7,321510739,9,-1,set,"[6463, 23352]"
2,7,27209511,20,-1,set,"[425, 2580, 5069, 6446, 11712]"
3,7,24774761,12,-1,set,"[5550, 5849, 5852]"
4,7,16686749,259,-1,set,"[53, 710, 835, 844, 1547, 3365, 3398, 3494, 43..."


In [164]:
# 测试用户信息数据集
test_user_info = user_info.head()
for index,row in  test_data.iterrows():
    test_user_info = test_user_info.append(user_info[user_info.mid == int(row.key)])
test_user_info

,id,mid,name,sex,sign,the_rank,level,jointime,moral,silence,birthday,coins,fans_badge,role,title,desc,vip_type,vip_status
0,1,17515174,试图让群主女装,NaN,NaN,10000.0,5,0.0,0.0,0.0,02-24,0.0,0.0,0.0,NaN,NaN,2.0,1.0
1,2,7596516,酒客小丑,NaN,沉迷于吃狗粮中。商业合作请加QQ2375432747 微博：@酒客小丑,10000.0,6,0.0,0.0,0.0,06-15,0.0,1.0,1.0,bilibili 2018百大UP主、高能联盟成员,NaN,2.0,1.0
2,3,10769575,文史阁,NaN,地势坤，君子以厚德载物。,10000.0,6,0.0,0.0,0.0,01-01,0.0,0.0,1.0,bilibili 知名科普UP主,代表作：中国历代帝王合集,0.0,0.0
3,4,22989278,果冻萌萌的,NaN,林正英以及香港经典鬼片僵尸片24小时播放 房间号：193 欢迎大家观看,10000.0,6,0.0,0.0,0.0,09-17,0.0,1.0,1.0,bilibili 知名UP主,NaN,1.0,0.0
4,5,130050712,初阳手绘石头,NaN,如果搜不到我，说明你是在查找你的好友里搜，当然搜不到了。因为我还不是你的好友呢，你要在添＋好...,10000.0,5,0.0,0.0,0.0,01-01,0.0,1.0,0.0,NaN,NaN,2.0,1.0
41850,41856,330817737,派大排,NaN,这个人懒死了，什么都没有写~o(〃'▽'〃)o,10000.0,3,0.0,0.0,0.0,10-11,0.0,0.0,0.0,NaN,NaN,0.0,0.0
30606,30613,321510739,狐妖枖仸殀,NaN,NaN,10000.0,4,0.0,0.0,0.0,11-30,0.0,0.0,0.0,NaN,NaN,1.0,0.0
5499,5499,27209511,frank不是什么大神啦,NaN,专注于鬼畜和生活，想要成为大剪辑师，加油，新粉丝群还是遇到状况了，具体开放时间请耐心等待～,10000.0,5,0.0,0.0,0.0,02-17,0.0,1.0,0.0,NaN,NaN,2.0,1.0
39740,39746,24774761,乐小昭,NaN,失踪人口一位…… 微博：@乐小昭,10000.0,3,0.0,0.0,0.0,01-01,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1443,1444,16686749,海棠花丶,NaN,NaN,10000.0,5,0.0,0.0,0.0,04-14,0.0,0.0,0.0,NaN,NaN,2.0,1.0


In [165]:
new = user_power(test_data,rules,test_user_info)
new

,db,key,size,ttl,type,value
0,7,330817737,6,-1,set,[132112]
1,7,330817737,6,-1,set,[132112]
2,7,321510739,9,-1,set,"[6463, 23352]"
3,7,321510739,9,-1,set,"[6463, 23352]"
4,7,321510739,9,-1,set,"[6463, 23352]"
5,7,321510739,9,-1,set,"[6463, 23352]"
6,7,27209511,20,-1,set,"[425, 2580, 5069, 6446, 11712]"
7,7,27209511,20,-1,set,"[425, 2580, 5069, 6446, 11712]"
8,7,27209511,20,-1,set,"[425, 2580, 5069, 6446, 11712]"
9,7,27209511,20,-1,set,"[425, 2580, 5069, 6446, 11712]"


可以看到，根据权重，相应的数据重复了多次